In [1]:
from google.colab import drive
drive.mount('/content/drive')
main_directory = '/content/drive/My Drive/Thesis/Thesis Codes 4_2/Dataset/train_new.csv'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from google.colab import drive

main_directory_t = '/content/drive/My Drive/Thesis/Thesis Codes 4_2/Dataset/test.csv'

In [0]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [4]:
train = pd.read_csv(main_directory,encoding = 'ISO-8859-1')
print(train)

        label                                                                                                            tweet
0           0  is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!
1           0                        @Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds
2           0                                                                  my whole body feels itchy and like its on fire 
3           0  @nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. 
4           0                                                                                    @Kwesidei not the whole crew 
...       ...                                                                                                              ...
155426      4                                                                         My GrandMa is making Dine

In [5]:
test = pd.read_csv(main_directory_t,encoding = 'ISO-8859-1',header=None,names =['label','id','date','query','user','tweet'])
print(test)

     label  ...                                                                                                                                         tweet
0        4  ...                               @stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.
1        4  ...                                                                                    Reading my kindle2...  Love it... Lee childs is good read.
2        4  ...                                                                                    Ok, first assesment of the #kindle2 ...it fucking rocks!!!
3        4  ...  @kenburbary You'll love your Kindle2. I've had mine for a few months and never looked back. The new big one is huge! No need for remorse! :)
4        4  ...                                                                   @mikefish  Fair enough. But i have the Kindle2 and I think it's perfect  :)
..     ...  ...                                     

In [6]:
train.shape, test.shape

((155431, 2), (498, 6))

In [7]:
train['label'].value_counts(normalize = True)

0    0.64767
4    0.35233
Name: label, dtype: float64

In [8]:
train.head()

,label,tweet
0,0,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
1,0,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
4,0,@Kwesidei not the whole crew


In [0]:
train['clean_tweet'] = train['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))
test['clean_tweet'] = test['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))



In [0]:
# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

train['clean_tweet'] = train['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

# convert text to lowercase
train['clean_tweet'] = train['clean_tweet'].str.lower()
test['clean_tweet'] = test['clean_tweet'].str.lower()

# remove numbers
train['clean_tweet'] = train['clean_tweet'].str.replace("[0-9]", " ")
test['clean_tweet'] = test['clean_tweet'].str.replace("[0-9]", " ")

# remove whitespaces
train['clean_tweet'] = train['clean_tweet'].apply(lambda x:' '.join(x.split()))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ' '.join(x.split()))

In [0]:
# import spaCy's language model
nlp = spacy.load('en', disable=['parser', 'ner'])

# function to lemmatize text
def lemmatization(texts):
    output = []
    for i in texts:
        s = [token.lemma_ for token in nlp(i)]
        output.append(' '.join(s))
    return output

In [0]:
train['clean_tweet'] = lemmatization(train['clean_tweet'])
test['clean_tweet'] = lemmatization(test['clean_tweet'])

In [14]:
train.sample(10)

,label,tweet,clean_tweet
84670,0,"Off to my little cousin's Christening, beautiful weather for it. Palacious reported out for the season grieving for his brother abroad","off to -PRON- little cousin 's christening , beautiful weather for -PRON- . palacious report out for the season grieve for -PRON- brother abroad"
22743,0,Checking outt,check outt
108326,4,"@mumbrella Premier's knew about it when it first launched, just took them ages to go onto it! Need I say anymore?","mumbrella premier 's know about -PRON- when -PRON- first launch , just take -PRON- age to go onto -PRON- need i say anymore"
113316,4,off to bed..did homework for 1 hr. 17 mins..and it's not even complete! and it's Math..garrr..test tomorrow! night,off to bed .. do homework for hr . min .. and -PRON- be not even complete and -PRON- be math .. garrr .. test tomorrow night
55494,0,So stressed out about finals next week! Can't believe I have to smoke to calm my nerves.,so stress out about final next week can not believe i have to smoke to calm -PRON- nerve .
56904,0,Ahhhhhhhhhhhj!!!!!!! All ive had is sushi all day!!!!!! And 1 bowl of total cereal!!! and I'm drinking vod water,ahhhhhhhhhhhj all -PRON- have have be sushi all day and bowl of total cereal and -PRON- be drink vod water
69666,0,"IIRC, all the starbucks I'll pass in Ft. Worth are closed already","iirc , all the starbuck -PRON- will pass in ft . worth be close already"
102370,4,@CASHUSCREAM hey there sexual chocolate caramel whip cream delight me &amp; @LatinaNichelle4 we're just talking about you lol,cashuscream hey there sexual chocolate caramel whip cream delight -PRON- amp latinanichelle -PRON- be just talk about -PRON- lol
22082,0,"Just bought new tax disc for my car. The first of many, many car related expenses this month","just buy new tax disc for -PRON- car . the first of many , many car relate expense this month"
71720,0,I'm at work for 4 hours even though its a bank holiday Will be home in time for the rest of the snooker though!,-PRON- be at work for hour even though -PRON- a bank holiday will be home in time for the rest of the snooker though


In [15]:
pip install "tensorflow>=1.7.0"

In [16]:
pip install tensorflow-hub

In [17]:
import tensorflow_hub as hub
import tensorflow as tf

In [0]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [19]:
x = ["Roasted ants are a popular snack in Columbia"]

# Extract ELMo features 
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

embeddings.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


TensorShape([Dimension(1), Dimension(8), Dimension(1024)])

In [0]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [0]:
list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

In [0]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['clean_tweet']) for x in list_train]


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [23]:
elmo_test = [elmo_vectors(x['clean_tweet']) for x in list_test]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


KeyboardInterrupt: ignored

In [0]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)


NameError: ignored

In [0]:
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [0]:
# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()



In [0]:
# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [0]:
# load elmo_train_new
pickle_in = open("elmo_train_03032019.pickle", "rb")
elmo_train_new = pickle.load(pickle_in)

# load elmo_train_new
pickle_in = open("elmo_test_03032019.pickle", "rb")
elmo_test_new = pickle.load(pickle_in)

In [0]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  train['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)

In [0]:
preds_valid = lreg.predict(xvalid)

In [0]:
f1_score(yvalid, preds_valid)

In [0]:
# make predictions on test set
preds_test = lreg.predict(elmo_test_new)

In [0]:
# prepare submission dataframe
sub = pd.DataFrame({'id':test['id'], 'label':preds_test})

# write predictions to a CSV file
sub.to_csv("sub_lreg.csv", index=False)